In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("building a warehouse")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
# read input
df = sqlContext.read.parquet("/algo/clicked-pageviews/2018/05/01/00/1525132800000_1525136400000_1525183032410")
publisher_subscribe_proba = {1023406, 1027019}

output_dfs = None
for publisher in publisher_subscribe_proba:
    # filter instances by publisher
    pub_df = df.filter(df['pv_publisherId'] == publisher)
    print 'publisher %d count %d' % (publisher, pub_df.count())

    if output_dfs == None:
        output_dfs = pub_df
    else:
        output_dfs = output_dfs.union(pub_df)

new_df = output_dfs.select('pv_userId', 'pv_publisherId', 'pv_userVisitedPublishers')
new_df.show()
new_df.write.format('json').save('/algo/publisher_embedding/input')
# new_df.write.save('/algo/publisher_embedding/input', format='parquet')

AnalysisException: u'Path does not exist: file:/algo/clicked-pageviews/2018/05/01/00/1525132800000_1525136400000_1525183032410;'

In [9]:
import pymysql.cursors
import pandas as pd
import ast

def query_MYSQL(query):
    result = []
    connection = pymysql.connect(host='backend-db9.taboolasyndication.com', # backend-db3 or crawler-db1
                             user='trc',
                             passwd='taboola',
                             db='trc',
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
    except Exception as error:
        print(error)
    finally:
        connection.close()  
    if len(result) == 0:
        return None
    else:
        return pd.DataFrame(result)

query = '''
    select pv.userId, pv.userVisitedPublishers
    from taboola_taz.pageviews_20180713
    '''
synd_language = query_MYSQL(query)
print(str(synd_language))

(1146, u"Table 'taboola_taz.pageviews_20180713' doesn't exist")
None


In [9]:
import pyspark
import logging
from ../../bigquery_io.bigquery_wrapper import BigQueryWrapper

logger = logging.getLogger(__name__)
from pyspark import SparkContext, SparkConf

bqw = BigQueryWrapper(1, spark_context=SparkContext(conf=SparkConf()))
test_query = """
select pv.requests.servedItems.syndicatorId as syndicatorId, pv.browserPlatform as platform,
    sum(pv.requests.servedItems.cpc*if (pv.requests.servedItems.clicked = True, 1, 0)) as spent,
    sum(if (pv.requests.servedItems.clicked = True, 1, 0)) as clicks,
    sum(if (pv.requests.servedItems.actionConversions.ppvAction = false, 1, 0)) as numActions,
    sum(if (pv.requests.servedItems.actionConversions.ppvAction = false, 1, 0))/sum(if (pv.requests.servedItems.clicked = True, 1, 0)) as cvr
    from taboola_taz.pageviews_20150905
    where pv.requests.servedItems.clicked = True
    group by syndicatorId,platform
    having  sum(if (pv.requests.servedItems.clicked = True, 1, 0))>1000 and numActions>0
    order by spent desc
"""
table_name_suffix = 'test'
stream = bqw.execute_sync_spark(query=test_query, table_name_suffix=table_name_suffix)
pass

SyntaxError: invalid syntax (<ipython-input-9-e9e3aea0c6e7>, line 3)

In [6]:
from bigquery import get_client

# BigQuery project id as listed in the Google Developers Console.
project_id = 'taboola-rowdata-bi'

# Service account email address as listed in the Google Developers Console.
service_account = 'algo-deeppersonalizationv2@taboola-rowdata-bi.iam.gserviceaccount.com'

# PKCS12 or PEM key provided by Google.
key = 'bigquery_io/bigquery_io/resources/key.pem'

client = get_client(project_id, service_account=service_account,
                    private_key_file=key, readonly=True)

# JSON key provided by Google
json_key = 'bigquery_io/bigquery_io/resources/algo-deep-personalization.json'
 
client = get_client(json_key_file=json_key, readonly=True)

# Submit an async query.
job_id, _results = client.query('SELECT userId, publisherId FROM [taboola-rowdata-bi:allen.subscription_model_input_20180723] LIMIT 1000')

# Check if the query has finished running.
complete, row_count = client.check_job(job_id)

# Retrieve the results.
results = client.get_query_rows(job_id)

Error: [('PEM routines', 'PEM_read_bio', 'no start line')]

In [3]:
print(results)

[{u'publisherId': 1068057, u'userId': u'868718d0d3e83687067bd2a4d50bce65d1ff1e02af63877658eef4285b51d087'}, {u'publisherId': 1031798, u'userId': u'306edd966b20ab380e1c5a5f86ef9157837efacde552c2a46486b1d460e93667'}, {u'publisherId': 1168, u'userId': u'1571182493c4cca49d5011899da1f3d3af189e0db95885f950a7cd87bb44ed40'}, {u'publisherId': 1089384, u'userId': u'f691d67b4c78c495875b080d6c893378e4b162a8fc3836bf8e21a95dbc9832ad'}, {u'publisherId': 1168, u'userId': u'1571182493c4cca49d5011899da1f3d3af189e0db95885f950a7cd87bb44ed40'}, {u'publisherId': 1168, u'userId': u'1571182493c4cca49d5011899da1f3d3af189e0db95885f950a7cd87bb44ed40'}, {u'publisherId': 1053057, u'userId': u'aba30678d3f384e610a08ad4d784e6dbf54c21013536949b5827ee1e7c1b823d'}, {u'publisherId': 1053057, u'userId': u'aba30678d3f384e610a08ad4d784e6dbf54c21013536949b5827ee1e7c1b823d'}, {u'publisherId': 1050363, u'userId': u'244c5e1d7a76c55f87e183c77310c7d44ec045c8cc64d036e6d3f5550ef59955'}, {u'publisherId': 1031851, u'userId': u'5087b0

# Reading dataframe using pyspark

In [1]:
import logging
import pyspark
import subprocess
import click
import traceback
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext

def _get_spark_handle():

    _spark = pyspark.sql.SparkSession.builder \
            .master("local[%d]" % 2) \
            .appName(
            "Spark Preprocessing") \
            .config("spark.executor.memory", "4g") \
            .config("spark.sql.execution.arrow.enable", "true") \
            .config("spark.ui.showConsoleProgress", "true") \
            .getOrCreate()

    return _spark


def _spark_handle():
    spark_handle = _get_spark_handle()
    return spark_handle


def _read_file(path):
    df = _spark_handle().read.options().parquet(path)
    return df


df = _read_file('hdfs://taz-hdfs.service.il.consul.taboolasyndication.com/algo/subscription/rawdata/wsj_pageviews/2018/07/26/part-00000-65a933cd-887b-4023-86b6-e4a453b9c61a-c000.snappy.parquet')
#df = _spark_handle().read.format("tfrecords").load('/Users/allen.wu/Desktop/data/weka_input/20180706/train/part-r-00000')
path = 'hdfs://taz-hdfs.service.il.consul.taboolasyndication.com/algo/subscription/training_input/start_date_2018-07-25_end_date_2018-07-25_pub_group_tag_subscription_model/train/part-r-00000'
#df = _spark_handle().read.format("tf").option("recordType", "Example").load(path)
df.groupby('is_positive').count().show()
output_dfs = df.select('pv_userId', 'pv_publisherId', 'is_positive', 'pv_pubSubscriptionActions', 'pv_userVisitedPublishers', 'pv_userSegmentsInfo_uddIds')
    
output_dfs.write.save('/Users/allen.wu/Desktop/data/weka_input/wsj', format='csv')
#read input
#df = sqlContext.read.parquet("hdfs://sam-il/production/page_views/2018/08/08/00/*")
#df.printSchema()
#df.select('pv_userVisitedPublishers').show()

+-----------+-----+
|is_positive|count|
+-----------+-----+
|          1|    3|
+-----------+-----+



AnalysisException: u'path file:/Users/allen.wu/Desktop/data/weka_input/wsj already exists.;'

# Merge csv files and output into 1

In [31]:
import os
from os import path

directory = "/Users/allen.wu/Desktop/data/weka_input/wsj/20180706_20180710/train_csv"
fw = open("/Users/allen.wu/Desktop/data/weka_input/wsj/20180706_20180710/train.csv", "w")
fw.write('country_code,os_name,os_family,session_depth,is_subscribed\n')
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        
        with open(os.path.join(directory, filename)) as f:
            fw.write((f.read()))
        f.close()
        continue
    else:
        continue
fw.close()